In [1]:
import numpy
import itertools
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from scipy import stats
from Bio.Alphabet import IUPAC
#from itertools import permutations
AA=["A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y"]

In [2]:
dipep = [''.join(i) for i in itertools.product(AA, repeat = 2)]

In [3]:
#print(dipep)

In [4]:

arr = numpy.empty((0,402), dtype=numpy.float)
class_arr = numpy.empty((0), dtype=numpy.int)
#print(arr.shape)
#print(arr)
all_fasta=("major_capsid.fasta","minor_capsid.fasta","baseplate.fasta",
          "major_tail.fasta","minor_tail.fasta","portal.fasta",
          "tail_fiber.fasta","tail_sheath.fasta","collar.fasta",
          "htj.fasta")
#print(all_fasta)
prot_class=0;
for file in all_fasta:
    print('####################' + file)
    for record in SeqIO.parse(file, "fasta"):
        #print(record.id)
        ll=len(record.seq)
        #xx=Seq(str(record.seq),IUPAC.protein)
        X = ProteinAnalysis(record.seq.__str__().replace('X','A').replace('J','L'))
        #X = ProteinAnalysis(str(xx))
        tt= [X.isoelectric_point(), X.instability_index()]
        tt_n = numpy.asarray(tt,dtype=numpy.float)

        dipep_count=[record.seq.count(i)/ll for i in dipep]
        dipep_count_n = numpy.asarray(dipep_count,dtype=numpy.float)
    
        cat_n= numpy.append(dipep_count_n,tt_n)
        cat_n = cat_n.reshape((1,cat_n.shape[0]))

        arr = numpy.append(arr,cat_n , axis=0)
        class_arr = numpy.append(class_arr,prot_class)
    prot_class+=1
    

    

####################major_capsid.fasta
####################minor_capsid.fasta
####################baseplate.fasta
####################major_tail.fasta
####################minor_tail.fasta
####################portal.fasta
####################tail_fiber.fasta
####################tail_sheath.fasta
####################collar.fasta
####################htj.fasta


In [5]:
 print(arr.shape)

#arr_z = numpy.apply_over_axis(stats.zscore,arr,0)
arr_z=numpy.apply_along_axis(stats.zscore,0,arr)
#arr_z=numpy.apply_along_axis(stats.zscore,0,arr)
print(arr_z)
print(arr)
print(arr_z.shape)
#print(prot_class)

(25675, 402)
[[ 0.57377186 -0.32328229  0.23723516 ... -0.54030715 -0.91371426
  -0.3992156 ]
 [-0.3978428  -0.32328229 -1.09643786 ... -0.54030715 -0.62372753
  -0.274854  ]
 [ 0.83157573 -0.32328229  0.96904548 ... -0.54030715 -0.82223744
  -1.16434231]
 ...
 [ 3.52473445 -0.32328229  2.70327206 ... -0.54030715  1.3824358
   0.80537142]
 [-1.0221569  -0.32328229 -1.09643786 ... -0.54030715 -0.5499655
  -0.27693521]
 [ 0.02712572 -0.32328229 -0.21958173 ...  0.14471935 -0.24993047
   0.24000084]]
[[1.32450331e-02 0.00000000e+00 6.62251656e-03 ... 0.00000000e+00
  4.59014893e+00 3.00536424e+01]
 [5.18134715e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  5.06573486e+00 3.12961399e+01]
 [1.53846154e-02 0.00000000e+00 1.02564103e-02 ... 0.00000000e+00
  4.74017334e+00 2.24092564e+01]
 ...
 [3.77358491e-02 0.00000000e+00 1.88679245e-02 ... 0.00000000e+00
  8.35589600e+00 4.20886792e+01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  5.18670654e+00 3.12753465e+0

In [6]:
mean_arr=numpy.apply_along_axis(numpy.mean,0,arr)
#print(mean_arr)
print(mean_arr.shape)
std_arr=numpy.apply_along_axis(numpy.std,0,arr)
print(std_arr.shape)

(402,)
(402,)


In [7]:
#targets = numpy.array([[2, 3, 4, 0]]).reshape(-1)

nb_classes = 10
one_hot_targets = numpy.eye(nb_classes)[class_arr]
print(one_hot_targets)
print(one_hot_targets.shape)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
(25675, 10)


In [12]:

final = numpy.concatenate((arr_z, one_hot_targets), axis=1)
final_raw = numpy.concatenate((arr, one_hot_targets), axis=1)

In [9]:
numpy.savetxt('test.out', final, delimiter='\t') 

In [13]:
numpy.savetxt('test2.out', final_raw, delimiter='\t') 

In [14]:
numpy.random.shuffle(final)
numpy.random.shuffle(final_raw)

In [17]:
import pickle
pickle.dump(final[0:24000,0:402], open( "dipep_new_train_x.p", "wb" ) )
pickle.dump(final[24000:,0:402],  open( "dipep_new_test_x.p" , "wb" ) )
pickle.dump(final[0:24000,402:], open( "dipep_new_train_y.p", "wb" ) )
pickle.dump(final[24000:,402:],  open( "dipep_new_test_y.p" , "wb" ) )
pickle.dump(mean_arr,  open( "dipep_new_mean.p" , "wb" ) )
pickle.dump(std_arr,  open( "dipep_new_std.p" , "wb" ) )

In [18]:
pickle.dump(final_raw[0:24000,0:402], open( "dipep_raw_new_train_x.p", "wb" ) )
pickle.dump(final_raw[24000:,0:402],  open( "dipep_raw_new_test_x.p" , "wb" ) )
pickle.dump(final_raw[0:24000,402:], open( "dipep_raw_new_train_y.p", "wb" ) )
pickle.dump(final_raw[24000:,402:],  open( "dipep_raw_new_test_y.p" , "wb" ) )